In [5]:
import psycopg2
import pandas as pd
import numpy as np
from psycopg2.extras import execute_values
import os

In [ ]:
# Database connection details
## get server login details
server_user = input(f'Enter username: ')
server_pass = input(f'Enter password: ')

## get database login details
DB_CONFIG = {
    "dbname": "Elections",
    "user": server_user,
    "password": server_pass,
    "host": "project-elections-dw-project-elections-dw.j.aivencloud.com",
    "port": "27706"
}

## Connect to the database
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

In [ ]:

def generate_ddl_from_file(file_path):
    """
    Generate SQL DDL from a CSV or Excel file
    
    Args:
        file_path (str): Path to the input file (.csv, .xls, or .xlsx)
    
    Returns:
        str: SQL DDL statement
    """
    
    # Extract the file extension using os.path.splitext
    _, ext = os.path.splitext(file_path)
    file_extension = ext.lower()[1:]  # Remove the leading dot

    if file_extension == 'csv':
        df = pd.read_csv(file_path)
    elif file_extension in ['xls', 'xlsx']:
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use .csv, .xls, or .xlsx files")

    dtype_mapping = {
        'object': 'TEXT',
        'int64': 'INTEGER',
        'float64': 'NUMERIC',
        'datetime64[ns]': 'TIMESTAMP',
        'bool': 'BOOLEAN'
    }
    
    # Use os.path.basename and os.path.splitext to extract the table name
    table_name = os.path.splitext(os.path.basename(file_path))[0].lower()
    columns = []
    for column, dtype in df.dtypes.items():
        sql_type = dtype_mapping.get(str(dtype), 'TEXT')
        columns.append(f'    "{column}" {sql_type}')
    
    ddl = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    ddl += ",\n".join(columns)
    ddl += "\n);"
    
    return df, ddl


In [ ]:
_, test_sql_ddl = generate_ddl_from_file('../OriginalMAIdata/ar_1999.xls')
print(test_sql_ddl)
ddl_path = './sql_scripts/ddl_script.sql'
with open(ddl_path, 'w') as f:
    f.write(test_sql_ddl)


CREATE TABLE IF NOT EXISTS ar_1999 (
    "CODIGO" INTEGER,
    "DIST_CONC_FREG" TEXT,
    "T_ELEI" TEXT,
    "D_ELEI" TEXT,
    "SIGLA" TEXT,
    "T_DADOS" TEXT,
    "VOTOS" INTEGER,
    "P_VOTOS" NUMERIC,
    "MANDATOS" INTEGER
);


In [ ]:
## Run SQL File
def run_sql_file(file_path):
    with open(file_path, 'r') as file:
        cur.execute(file.read())
run_sql_file('dimension_tables_init.sql')